In [36]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt

dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])

In [37]:
df[df['salary_year_avg'].notna()]['salary_year_avg']

28        109500.0
77        140000.0
92        120000.0
100       228222.0
109        89000.0
            ...   
785624    139216.0
785641    150000.0
785648    221875.0
785682    157500.0
785692    157500.0
Name: salary_year_avg, Length: 22003, dtype: float64

In [38]:
help(df.apply)

Help on method apply in module pandas.core.frame:

apply(func: 'AggFuncType', axis: 'Axis' = 0, raw: 'bool' = False, result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None, args=(), by_row: "Literal[False, 'compat']" = 'compat', engine: "Literal['python', 'numba']" = 'python', engine_kwargs: 'dict[str, bool] | None' = None, **kwargs) method of pandas.core.frame.DataFrame instance
    Apply a function along an axis of the DataFrame.
    
    Objects passed to the function are Series objects whose index is
    either the DataFrame's index (``axis=0``) or the DataFrame's columns
    (``axis=1``). By default (``result_type=None``), the final return type
    is inferred from the return type of the applied function. Otherwise,
    it depends on the `result_type` argument.
    
    Parameters
    ----------
    func : function
        Function to apply to each column or row.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis along which the function is applied:
 

In [39]:
df_salary = df[df['salary_year_avg'].notna()].copy()

def projected_salary(salary):
    return salary * 1.03

df_salary['salary_year_inflated'] = df_salary['salary_year_avg'].apply(projected_salary)

df_salary[['salary_year_avg','salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


In [40]:
# using lambda function

df_salary['salary_year_inflated'] = df['salary_year_avg'].apply(lambda salary: salary*1.03)

df_salary[['salary_year_avg','salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


In [41]:
# another way - simple way since we are just multiplying 1.03

df_salary['salary_year_inflated'] = df['salary_year_avg'] * 1.03

df_salary[['salary_year_avg','salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


## Converting string column to list column

In [42]:
df['job_skills'][1]

"['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']"

In [43]:
type(df['job_skills'][1])   # instead of list, it is a string

str

In [44]:
list(df['job_skills'][1])    # wrapping it in list function doesn't work

['[',
 "'",
 'r',
 "'",
 ',',
 ' ',
 "'",
 'p',
 'y',
 't',
 'h',
 'o',
 'n',
 "'",
 ',',
 ' ',
 "'",
 's',
 'q',
 'l',
 "'",
 ',',
 ' ',
 "'",
 'n',
 'o',
 's',
 'q',
 'l',
 "'",
 ',',
 ' ',
 "'",
 'p',
 'o',
 'w',
 'e',
 'r',
 ' ',
 'b',
 'i',
 "'",
 ',',
 ' ',
 "'",
 't',
 'a',
 'b',
 'l',
 'e',
 'a',
 'u',
 "'",
 ']']

In [45]:
import ast

ast.literal_eval(df['job_skills'][1])

['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']

In [46]:
type(ast.literal_eval(df['job_skills'][1]))

list

In [47]:
def clean_list(skill_list):
    return ast.literal_eval(skill_list)

df['job_skills'] = df['job_skills'].apply(clean_list)

ValueError: malformed node or string: None

In [48]:
# we have an error due to None values present in the column

df[df['job_skills'].isna()]['job_skills']

0         None
21        None
26        None
29        None
36        None
          ... 
785713    None
785715    None
785718    None
785724    None
785729    None
Name: job_skills, Length: 117037, dtype: object

In [30]:
# so we need to modify our function to not operate on None values

def clean_list(skill_list):
    if pd.notna(skill_list):
        return ast.literal_eval(skill_list)

df['job_skills'] = df['job_skills'].apply(clean_list)

In [50]:
df['job_skills'][1]

['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']

In [52]:
type(df['job_skills'][1])   # converted string to list now

list

In [ ]:
# using lambda function for the same operation


# df['job_skills'] = df['job_skills'].apply(lambda skill_list: ast.literal_eval(skill_list) if pd.notna(skill_list) else skill_list)

## Calulate Projected Salary next year

*   Senior roles assume 5%
*   Other roles assume 3%

In [53]:
# we need to use apply() with a condition now

help(df.apply)

Help on method apply in module pandas.core.frame:

apply(func: 'AggFuncType', axis: 'Axis' = 0, raw: 'bool' = False, result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None, args=(), by_row: "Literal[False, 'compat']" = 'compat', engine: "Literal['python', 'numba']" = 'python', engine_kwargs: 'dict[str, bool] | None' = None, **kwargs) method of pandas.core.frame.DataFrame instance
    Apply a function along an axis of the DataFrame.
    
    Objects passed to the function are Series objects whose index is
    either the DataFrame's index (``axis=0``) or the DataFrame's columns
    (``axis=1``). By default (``result_type=None``), the final return type
    is inferred from the return type of the applied function. Otherwise,
    it depends on the `result_type` argument.
    
    Parameters
    ----------
    func : function
        Function to apply to each column or row.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis along which the function is applied:
 

In [54]:
df_salary.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills,salary_year_inflated
28,Data Scientist,CRM Data Specialist,"San José Province, San José, Costa Rica",via Ai-Jobs.net,Full-time,False,Costa Rica,2023-08-01 13:37:57,False,False,Costa Rica,year,109500.0,NaN,Netskope,"['gdpr', 'excel']","{'analyst_tools': ['excel'], 'libraries': ['gd...",112785.00
77,Data Engineer,Data Engineer,"Arlington, VA",via LinkedIn,Full-time,False,Sudan,2023-06-26 14:22:54,False,False,Sudan,year,140000.0,NaN,Intelletec,"['mongodb', 'mongodb', 'python', 'r', 'sql', '...","{'analyst_tools': ['tableau'], 'cloud': ['orac...",144200.00
92,Data Engineer,Remote - Data Engineer - Permanent - W2,Anywhere,via LinkedIn,Full-time,True,"Illinois, United States",2023-02-21 13:29:59,False,True,United States,year,120000.0,NaN,Apex Systems,"['sql', 'python']","{'programming': ['sql', 'python']}",123600.00
100,Data Scientist,"Data Scientist, Risk Data Mining - USDS","Mountain View, CA",via LinkedIn,Full-time,False,"California, United States",2023-07-31 13:01:18,False,True,United States,year,228222.0,NaN,TikTok,"['sql', 'r', 'python', 'express']","{'programming': ['sql', 'r', 'python'], 'webfr...",235068.66
109,Data Analyst,Senior Supply Chain Analytics Analyst,Anywhere,via Get.It,Full-time,True,"Illinois, United States",2023-10-12 13:02:19,False,True,United States,year,89000.0,NaN,Get It Recruit - Transportation,"['python', 'r', 'alteryx', 'tableau']","{'analyst_tools': ['alteryx', 'tableau'], 'pro...",91670.00


In [58]:
# now we need to apply the apply() function to all rows (instead of all columns like we did before); we need to use axis=1

def projected_salary_new(row):
    if "Senior" in row['job_title_short']:
        return row['salary_year_avg']*1.05
    else:
        return row['salary_year_avg']*1.03

df_salary['salary_year_inf_new'] = df_salary.apply(projected_salary_new, axis = 1)

df_salary[['job_title_short','salary_year_avg', 'salary_year_inf_new']].sample(20)

,job_title_short,salary_year_avg,salary_year_inf_new
262006,Data Analyst,125000.0,128750.00
561875,Senior Data Scientist,175000.0,183750.00
325303,Data Scientist,215000.0,221450.00
658514,Senior Data Engineer,125000.0,131250.00
24859,Data Analyst,50000.0,51500.00
257052,Data Scientist,125000.0,128750.00
593587,Data Analyst,83500.0,86005.00
588975,Senior Data Scientist,129500.0,135975.00
645556,Data Analyst,125000.0,128750.00
688401,Data Engineer,147500.0,151925.00


In [60]:
# lambda function alternative

df_salary['salary_inf_lambda'] = df_salary.apply(lambda row: row['salary_year_avg']*1.05 if "Senior" in row['job_title_short'] else row['salary_year_avg']*1.03, axis = 1)

df_salary[['job_title_short','salary_year_avg', 'salary_year_inf_new', 'salary_inf_lambda']].sample(20)

,job_title_short,salary_year_avg,salary_year_inf_new,salary_inf_lambda
333810,Data Engineer,155500.0,160165.000,160165.000
233908,Senior Data Engineer,150000.0,157500.000,157500.000
445333,Senior Data Analyst,112100.0,117705.000,117705.000
666059,Data Engineer,375000.0,386250.000,386250.000
190000,Machine Learning Engineer,50400.0,51912.000,51912.000
180765,Business Analyst,105515.0,108680.450,108680.450
411518,Senior Data Scientist,180000.0,189000.000,189000.000
114843,Data Analyst,79930.0,82327.900,82327.900
507217,Senior Data Scientist,156000.0,163800.000,163800.000
456997,Data Analyst,107025.0,110235.750,110235.750
